In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("D:/Duong-Desktop/Code AI/digit-recognizer/train.csv") 
test_data = pd.read_csv("D:/Duong-Desktop/Code AI/digit-recognizer/test.csv")

In [3]:
x_train = train_data.drop("label", axis = 1)
y_train = train_data["label"]
x_train = x_train.values.reshape(-1, 1, 28, 28).astype(np.float32)  # Add channel dimension
x_train = x_train / 255.0  # Normalize the pixel values (scale to [0, 1])

In [4]:
import torch.utils.data
x_train_tensor = torch.tensor(x_train)
y_train_tensor = torch.tensor(y_train)
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,6))
ax[0].plot(x_train.values[9])
ax[0].set_title('784x1 data')
ax[1].imshow(x_train.values[9].reshape(28,28), cmap='gray')
ax[1].set_title('28x28 data')

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # First Block
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # Conv2D layer
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),  # Conv2D layer
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2, stride=2),  # MaxPool2D layer
            nn.Dropout(0.25)  # Dropout layer
        )

        # Second Block
        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # Conv2D layer
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),  # Conv2D layer
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),  # MaxPool2D layer
            nn.Dropout(0.25)  # Dropout layer
        )

        # Fully Connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 7 * 7, 512),  # Linear layer (assuming input images are 28x28)
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 1024),  # Linear layer
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 10),  # Output layer with 10 classes
            nn.Softmax(dim=1)  # Softmax for multi-class classification
        )
    
    

In [18]:
model = CNNModel()

In [19]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 5. Training Loop
num_epochs = 1  # You can adjust this based on your needs

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        print(outputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Optimize
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

RuntimeError: Given groups=1, weight of size [16, 784, 3, 3], expected input[64, 1, 28, 28] to have 784 channels, but got 1 channels instead

In [12]:
submission = pd.read_csv("D:/Duong-Desktop/Code AI/digit-recognizer/sample_submission.csv")

test_data = test_data.values.reshape(-1, 1, 28, 28).astype(np.float32)  # Add channel dimension
test_data = test_data / 255.0  # Normalize the pixel values (scale to [0, 1])

submission_tensor = torch.tensor(test_data)
answer = model(submission_tensor)

In [14]:
submission = pd.read_csv("D:/Duong-Desktop/Code AI/digit-recognizer/sample_submission.csv")
submission["Label"] = answer.detach()

In [15]:
submission.to_csv("submission.csv", index = False)